### Imports


In [20]:
%%capture
import os
import sys
!{sys.executable} -m pip install csv2md

### Benchmarking optimized/unoptimized with memory usage
drop valid .zok files in files dir and run notebook. Results will be saved in exports/data/result.csv

In [26]:
%%capture
compile_optimized = []
constraints_optimized = []
compile_unoptimized = []
constraints_unoptimized = []

mem_usg = []

%alias_magic benchmark timeit -p "-n 1 -r 1 -o"

# empties files from previous results
def reset_files():
    results = open("exports/data/result.csv", "w")
    results.write("compile_opt_μs, constr_opt, memusg_opt_kb, compile_unopt_μs, constr_unopt, memusg_unopt_kb\n")
    open("exports/data/memusg_res.txt", 'w').close()
    
# compiles with locally installed zokrates with optimizer active
def compile_opti(file):
    cmd = f"./memusg.sh zokrates compile -i files/{file}"
    value = %benchmark !{cmd}
    return int(value.best * 1000000)

# compiles without optimizer
def compile_unopti(file):
    cmd = f"./memusg.sh ./zokrates_unoptimized compile -i files/{file}"
    value = %benchmark !{cmd}
    return int(value.best * 1000000)
    
def get_memusg():
    with open('exports/data/memusg_res.txt') as f:
        return [line.rstrip() for line in f]
    
# counts constraints by looking into out.code
def count_constraints():
    f = open("./out.ztf",'r')
    return len(list(filter(lambda x: "==" in x, f.readlines())))
    
def export_data():
    file = open("exports/data/result.csv", "a")
    for i, val in enumerate(compile_optimized): 
        file.write(
            str(val) + ", " + 
            str(constraints_optimized[i]) + ", " + 
            str(mem_usg[i * 2]) + ", " + 
            str(compile_unoptimized[i]) + ", " + 
            str(constraints_unoptimized[i]) + ", " + 
            str(mem_usg[(i * 2) + 1]) + "\n"
        )
    file.close()

reset_files()
for file in os.listdir('./files'):
    compile_optimized.append(compile_opti(file))
    constraints_optimized.append(count_constraints())
    compile_unoptimized.append(compile_unopti(file))
    constraints_unoptimized.append(count_constraints())


mem_usg = get_memusg()
export_data()

In [27]:
!csv2md exports/data/result.csv > exports/data/table.md

with open("exports/data/table.md") as f:
    print(f.read())

| compile_opt_μs |  constr_opt |  memusg_opt_kb |  compile_unopt_μs |  constr_unopt |  memusg_unopt_kb |
| -------------- | ----------- | -------------- | ----------------- | ------------- | ---------------- |
| 20281488       |  232537     |  396664        |  16558760         |  257213       |  426184          |
| 5084822        |  56984      |  140728        |  3890349          |  63136        |  138492          |

